Create_CoughViD_images.ipynb
Creation of images from audio files.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

folder = '/content/drive/My Drive/Colab Notebooks/coughvid'
if not os.path.exists(folder):
  print(folder + ' does not exist')
data_folder = '/content/drive/My Drive/Colab Notebooks/coughvid/public_dataset'
if not os.path.exists(data_folder):
  print(data_folder + ' does not exist')

In [3]:
import pandas as pd
import os
import librosa
import librosa.display
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

Read in dataframe created previously.

In [4]:
df_coughvid = pd.read_csv(os.path.join(folder, 'df_coughvid.csv'))
df_coughvid.head(10)

,uuid,cough_detected,age,gender,status,audio_file,image_file
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,00039425-7f3a-42aa-ac13-834aaa2b6b92.png
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.png
2,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.png
3,001e2f19-d81c-4029-b33c-d2db56b23a4a,0.7811,20.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001e2f19-d81c-4029-b33c-d2db56b23a4a.png
4,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0.8937,28.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952.png
5,00291cce-36a0-4a29-9e2d-c1d96ca17242,0.9883,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,00291cce-36a0-4a29-9e2d-c1d96ca17242.png
6,002d28bc-7806-4dfb-9c9b-afa8cb623cac,0.9959,46.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,002d28bc-7806-4dfb-9c9b-afa8cb623cac.png
7,0037f67c-0d6f-42e6-ab94-66499d4d2bf4,0.8109,39.0,other,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0037f67c-0d6f-42e6-ab94-66499d4d2bf4.png
8,003e1bf6-2ecf-4a00-9198-29db34e8d7ac,0.9794,51.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,003e1bf6-2ecf-4a00-9198-29db34e8d7ac.png
9,0044964d-f7e9-43e3-831f-ae3e6374a9a4,0.9664,42.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0044964d-f7e9-43e3-831f-ae3e6374a9a4.png


Add path to audio_file column.




In [ ]:
def add_path(row):
  row.audio_file = data_folder +'/' + row.audio_file
  return row
df_coughvid = df_coughvid.apply(add_path, axis = 'columns')
df_coughvid.head(10)

,uuid,cough_detected,age,gender,status,audio_file
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
2,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
3,001e2f19-d81c-4029-b33c-d2db56b23a4a,0.7811,20.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
4,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0.8937,28.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
5,00291cce-36a0-4a29-9e2d-c1d96ca17242,0.9883,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
6,002d28bc-7806-4dfb-9c9b-afa8cb623cac,0.9959,46.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
7,0037f67c-0d6f-42e6-ab94-66499d4d2bf4,0.8109,39.0,other,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
8,003e1bf6-2ecf-4a00-9198-29db34e8d7ac,0.9794,51.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...
9,0044964d-f7e9-43e3-831f-ae3e6374a9a4,0.9664,42.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...


In [ ]:
print(df_coughvid.audio_file[0])
print(df_coughvid.audio_file[5])

/content/drive/My Drive/Colab Notebooks/coughvid/public_dataset/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm
/content/drive/My Drive/Colab Notebooks/coughvid/public_dataset/00291cce-36a0-4a29-9e2d-c1d96ca17242.webm


Use code from virufy.com to change audio files into image files.
(I have made changes to reflect my variable names and I have not split into test and train yet which they had done with their data.)

In [ ]:
# Functions to process audio files into images (adapted from code at virufy.com)
def trim_silence(x, *args):
    try:pad,db_max,frame_length,hop_length = args[0],args[1],args[2],args[3]
    except: 
        print('Please enter the following arguments: pad,db_max,frame_length,hop_length')
        return

    _, ints = librosa.effects.trim(x, top_db=db_max, frame_length=256, hop_length=64)
    start = int(max(ints[0]-pad, 0))
    end   = int(min(ints[1]+pad, len(x)))
    return x[start:end]

def process_cough_file(path,trim,*args):
    try: sr,removeaudio,chunk,db_max = args[0],args[1],args[2],args[3]
    except: 
        sr,removeaudio,chunk,db_max= 48000,False,3,50
    try:
        x,sr = librosa.load(path, sr=sr)       
    except: 
        return -1
    
    if len(x)/sr < 0.3 or len(x)/sr > 30:
        return None,None  
    hop_length = np.floor(0.010*sr).astype(int) #10ms
    win_length = np.floor(0.020*sr).astype(int) #20ms  

    if removeaudio:
        os.remove(path)
    
    x = trim(x, 0.25*sr, db_max,win_length,hop_length) 
    x = x[:np.floor(chunk*sr).astype(int)]
    
    #pads to chunk size if smaller
    x_pad = np.zeros(int(sr*chunk))
    x_pad[:min(len(x_pad), len(x))] = x[:min(len(x_pad), len(x))]

    return [x_pad,sr,hop_length,win_length]

def get_melspec(sdir,audio,sr,name):
    #Mel Spectogram
    plt.ioff()
    fig      = plt.figure()
    melspec  = librosa.feature.melspectrogram(y=audio,sr=sr)
    s_db     = librosa.power_to_db(melspec, ref=np.max)
    librosa.display.specshow(s_db)
    fig.canvas.draw()
    img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig=fig)
    #img = img[80:250,80:300]

    savepath = os.path.join(sdir,name+'.png') # Currently saving melspectrogram images to the folders specified in extract features
    cv2.imwrite(savepath,img)
    return savepath

def get_rawMFCCs(audio,sr,*args):
    try: hop_length,win_length,n_mfcc,n_mels,n_ftt = args[0],args[1],args[2],args[3],args[4]
    except:
        hop_length = np.floor(0.010*sr).astype(int) #10ms
        win_length = np.floor(0.020*sr).astype(int) #20ms  
        n_mfcc,n_mels,n_ftt=13,13,2048

    rawMFCCs    = librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=n_mfcc,n_mels=n_mels, n_fft=n_ftt, hop_length=hop_length)
    rawMFCCs    = np.mean(rawMFCCs.T,axis=0).tolist()

    return rawMFCCs

def getlabel(key, dataframe, chosen):
      return dataframe.loc[dataframe[chosen['id']]==key][chosen['status']].tolist()[0]

def extract(df, chosen, savedir):
    if not os.path.isdir(savedir):
        os.mkdir(savedir)
        
    keys, dirs = df[chosen['id']].tolist(),df[chosen['path']].tolist()  
    audio_objs = [process_cough_file(path,trim_silence) for path in dirs]
    false_indices = [i for i in range(len(audio_objs)) if isinstance(audio_objs[i],int) or isinstance(audio_objs[i],tuple)]

    audio_objs = [audio_objs[i] for i in range(len(audio_objs)) if i not in false_indices]
    audio_objs = np.array(audio_objs)
    audio,sr,hop_length,win_length = audio_objs[:,0],audio_objs[:,1],audio_objs[:,2],audio_objs[:,3]
    
    dirs = [dirs[i] for i in range(len(dirs)) if i not in false_indices]
    keys = [keys[i] for i in range(len(keys)) if i not in false_indices]
    data = {key:{'DIR':get_melspec(savedir,a_i,sr_i,key),
             'rawMFCC':get_rawMFCCs(a_i,sr_i),
             'label':getlabel(key, df, chosen)} for key,a_i,sr_i in list(zip(keys,audio,sr))}
    return data

def filter_DF(df):
    names = list(df.columns)
    chosen= {}
    for name in names:
        if 'status' in name.lower():chosen['status'] = name # Choosing the target (pcr_test_result_inferred)
        elif 'audio_file' in name.lower():chosen['path'] = name
        elif 'uuid' in name.lower() or 'id' == name.lower() :chosen['id'] = name
    return df[[chosen['id'],chosen['status'],chosen['path']]].dropna().reset_index(), chosen 

def create_images(df):
    dataframe, chosen = filter_DF(df)
    features = extract(dataframe, chosen, (folder + '/coughvid_images/'))
    return features

Processing the audio files into images.
Google crashes with too many files at once so I have processed them in batches of 1000.

In [ ]:
features = create_images(df_coughvid.iloc[:5])

In [ ]:
features = create_images(df_coughvid.iloc[5:1000])

In [ ]:
features = create_images(df_coughvid.iloc[1000:2000])

In [ ]:
features = create_images(df_coughvid.iloc[2000:3000])

In [ ]:
features = create_images(df_coughvid.iloc[3000:4000])

In [ ]:
features = create_images(df_coughvid.iloc[4000:5000])

In [ ]:
features = create_images(df_coughvid.iloc[5000:6000])

In [ ]:
features = create_images(df_coughvid.iloc[6000:7000])

In [ ]:
features = create_images(df_coughvid.iloc[7000:8000])

In [ ]:
features = create_images(df_coughvid.iloc[8000:9000])

In [ ]:
features = create_images(df_coughvid.iloc[9000:10000])

In [ ]:
features = create_images(df_coughvid.iloc[10000:])

Create dataframe of new image files.

In [ ]:
fnames = [fname for fname in os.listdir(folder + '/coughvid_images/')]

df_fnames = pd.DataFrame(columns =['uuid','image_file'])

for fname in fnames:
  split = fname.split(".")
  uuid = split[0]
  df_fnames = df_fnames.append({'uuid':uuid, 'image_file':fname}, ignore_index=True)
print(df_fnames.shape)
print(df_coughvid.shape)
df_fnames.head()

(10556, 2)
(10556, 6)


,uuid,image_file
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,00039425-7f3a-42aa-ac13-834aaa2b6b92.png
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.png
2,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.png
3,001e2f19-d81c-4029-b33c-d2db56b23a4a,001e2f19-d81c-4029-b33c-d2db56b23a4a.png
4,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952.png


Add images to df_coughvid dataframe.

In [ ]:
left = df_coughvid.set_index(['uuid'])
right = df_fnames.set_index(['uuid'])
df_coughvid = left.join(right)
print(df_coughvid.shape)
df_coughvid.head()

(10556, 6)


,cough_detected,age,gender,status,audio_file,image_file
uuid,,,,,,
00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,00039425-7f3a-42aa-ac13-834aaa2b6b92.png
0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.png
001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.png
001e2f19-d81c-4029-b33c-d2db56b23a4a,0.7811,20.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001e2f19-d81c-4029-b33c-d2db56b23a4a.png
0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0.8937,28.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952.png


Save df_coughvid dataframe ready to use in the next stage.

In [ ]:
folder = '/content/drive/My Drive/Colab Notebooks/coughvid'
df_coughvid.to_csv(os.path.join(folder, 'df_coughvid.csv'))

Check file has saved correctly.

In [ ]:
df_coughvid = pd.read_csv(os.path.join(folder, 'df_coughvid.csv'))
df_coughvid.head(10)

,uuid,cough_detected,age,gender,status,audio_file,image_file
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,00039425-7f3a-42aa-ac13-834aaa2b6b92.png
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.png
2,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.png
3,001e2f19-d81c-4029-b33c-d2db56b23a4a,0.7811,20.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,001e2f19-d81c-4029-b33c-d2db56b23a4a.png
4,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0.8937,28.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952.png
5,00291cce-36a0-4a29-9e2d-c1d96ca17242,0.9883,15.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,00291cce-36a0-4a29-9e2d-c1d96ca17242.png
6,002d28bc-7806-4dfb-9c9b-afa8cb623cac,0.9959,46.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,002d28bc-7806-4dfb-9c9b-afa8cb623cac.png
7,0037f67c-0d6f-42e6-ab94-66499d4d2bf4,0.8109,39.0,other,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0037f67c-0d6f-42e6-ab94-66499d4d2bf4.png
8,003e1bf6-2ecf-4a00-9198-29db34e8d7ac,0.9794,51.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,003e1bf6-2ecf-4a00-9198-29db34e8d7ac.png
9,0044964d-f7e9-43e3-831f-ae3e6374a9a4,0.9664,42.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,0044964d-f7e9-43e3-831f-ae3e6374a9a4.png


In [ ]:
df_coughvid.shape

(10556, 7)